In [1]:
import numpy as np
import pandas as pd
import struct
import math
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import seaborn as sns
sns.set_style('darkgrid')
import sys
import os

In [16]:
#mu = [0, 10, 20, 30, 40]
#file_number = [99, 99, 99, 99, 99]
#eig_number = [100, 100, 100, 100, 100]
mu = [0, 5, 25, 40]
file_number = [700, 600, 500, 420]
eig_number = [60, 60, 60, 60]

data_arr = []
#arr1 = np.empty(4)
smearing = 'smeared'
path = '/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/CSV/eigenvalues_mu.csv'

for t in range (len(mu)):
    w=mu[t]//10
    q=mu[t]-w*10
    for i in range (1, file_number[t]+1):
        a=i//1000
        b=(i-a*1000)//100
        c=(i-a*1000-b*100)//10
        d=(i-a*1000-b*100-c*10)
        if os.path.isfile('/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/time_32/mu0.%d%d/eigenvalues_%d%d%d%d' % (w, q, a, b, c, d)):
            with open('/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/time_32/mu0.%d%d/eigenvalues_%d%d%d%d' % (w, q, a, b, c, d), "rb") as f:
                byte = f.read(8*eig_number[t]*2)
                str1 = struct.unpack("d"*eig_number[t]*2, byte)
                for k in range(0, eig_number[t]):
                    data_arr.append([mu[t]/100, i, str1[2*k], str1[2*k+1]])
df = pd.DataFrame(np.array(data_arr), columns=['mu', 'conf_number', 're', 'im'])
print(df)
df.to_csv(path, index=False)

         mu  conf_number        re        im
0       0.0          1.0  0.007500 -0.001230
1       0.0          1.0  0.007500  0.001230
2       0.0          1.0  0.007500 -0.001230
3       0.0          1.0  0.007500  0.001230
4       0.0          1.0  0.007500 -0.025057
5       0.0          1.0  0.007500  0.025057
6       0.0          1.0  0.007500  0.025057
7       0.0          1.0  0.007500 -0.025057
8       0.0          1.0  0.007500 -0.026069
9       0.0          1.0  0.007500  0.026069
10      0.0          1.0  0.007500 -0.026069
11      0.0          1.0  0.007500  0.026069
12      0.0          1.0  0.007500  0.033522
13      0.0          1.0  0.007500 -0.033522
14      0.0          1.0  0.007500 -0.033522
15      0.0          1.0  0.007500  0.033522
16      0.0          1.0  0.007500  0.034583
17      0.0          1.0  0.007500 -0.034583
18      0.0          1.0  0.007500 -0.034583
19      0.0          1.0  0.007500  0.034583
20      0.0          1.0  0.007500  0.038203
21      0.

In [2]:
T = [32, 24, 16, 8]
file_number = [2000, 320, 160, 200]
eig_number = [60, 100, 160, 200]
#file_number = [99, 320, 160, 200]
#eig_number = [100, 100, 200, 160]


data_arr = []
arr1 = np.empty(4)
smearing = 'smeared'
path = '/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/CSV/eigenvalues_T.csv'

for t in range (len(T)):
    for i in range (1, file_number[t]+1):
        a=i//1000
        b=(i-a*1000)//100
        c=(i-a*1000-b*100)//10
        d=(i-a*1000-b*100-c*10)
        if os.path.isfile('/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/time_%d/mu0.00/eigenvalues_%d%d%d%d' % (T[t], a, b, c, d)):
            with open('/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/time_%d/mu0.00/eigenvalues_%d%d%d%d' % (T[t], a, b, c, d), "rb") as f:
                byte = f.read(8*eig_number[t]*2)
                str1 = struct.unpack("d"*eig_number[t]*2, byte)
                for k in range(0, eig_number[t]):
                    data_arr.append([T[t], i, str1[2*k], str1[2*k+1]])
df = pd.DataFrame(np.array(data_arr), columns=['T', 'conf_number', 're', 'im'])
print(df)
df.to_csv(path, index=False)

           T  conf_number      re        im
0       32.0          1.0  0.0075  0.002298
1       32.0          1.0  0.0075  0.002298
2       32.0          1.0  0.0075 -0.002298
3       32.0          1.0  0.0075 -0.002298
4       32.0          1.0  0.0075  0.010006
...      ...          ...     ...       ...
215595   8.0        200.0  0.0075 -0.271474
215596   8.0        200.0  0.0075  0.271474
215597   8.0        200.0  0.0075 -0.271949
215598   8.0        200.0  0.0075  0.271949
215599   8.0        200.0  0.0075  0.272574

[215600 rows x 4 columns]


In [2]:
#functions for extracting unique positive slepc eigenvalues
def read_file(filepath):
    line = []
    eigs = []
    with open(filepath) as fp:
        lines = fp.readlines()
        for i in range(1, len(lines)-1):
            eigs.append(complex(lines[i][:-2] + 'j'))
    return eigs

def compare(a, b, tol):
    ident = 0
    if abs(abs(a.real) - abs(b.real)) < tol and abs(abs(a.imag) - abs(b.imag)) < tol:
        ident = 1
    return ident

def make_unique(eigs, tol):
    eigs1 = []
    ident = 0
    eigs1.append(complex(abs(eigs[0].real), abs(eigs[0].imag)))
    for i in range(len(eigs)):
        for j in range(len(eigs1)):
            if compare(eigs[i], eigs1[j], tol) == 1:
                ident = 1
        if(ident == 0):
            eigs1.append(complex(abs(eigs[i].real), abs(eigs[i].imag)))
        ident = 0
    return eigs1

eigs = read_file('../../result/data/smeared/time_32/mu0.00/eigenvalues_eig=60_0001')

eigs_new = make_unique(eigs, 1e-6)
for i in range(len(eigs_new)):
    print(eigs_new[i])

(0.007500000000000071+0.0012304018213407089j)
(0.007499999999999915+0.025057053089106555j)
(0.007499999999999993+0.02606932821876798j)
(0.007500000000000062+0.03352247693934661j)
(0.007499999999999923+0.034582525291206145j)
(0.007499999999999992+0.038202851563831845j)
(0.007500000000000041+0.040573168452031995j)
(0.007499999999999918+0.04520842197537144j)
(0.007499999999999936+0.04728101414192484j)
(0.007500000000000015+0.05011547695755184j)
(0.007500000000000013+0.0527271591994573j)
(0.007499999999999955+0.055103380678064355j)
(0.007500000000000082+0.05627620766905213j)
(0.0074999999999999095+0.059281404121245475j)
(0.007500000000000069+0.06096534122113409j)


In [3]:
#creating CVS from smeared slepc eigenvalues for different mu
#mu = [0, 3, 5, 10, 25]
#file_number = [120, 55, 55, 100, 100]
mu=[0]
file_number = [300]
path = '../../result/data/smeared/CSV/eigenvalues_mu.csv'
tol = 1e-6

data_arr = []

for t in range (len(mu)):
    w=mu[t]//10
    q=mu[t]-w*10
    for i in range (1, file_number[t]+1):
        a=i//1000
        b=(i-a*1000)//100
        c=(i-a*1000-b*100)//10
        d=(i-a*1000-b*100-c*10)
        if os.path.isfile('../../result/data/smeared/time_32/mu0.%d%d/eigenvalues_double_interval=0-0.3_eig=60_%d%d%d%d' % (w, q, a, b, c, d)):
            eigs = read_file('../../result/data/smeared/time_32/mu0.%d%d/eigenvalues_double_interval=0-0.3_eig=60_%d%d%d%d' % (w, q, a, b, c, d))
            #print(i)
            eigs_new = make_unique(eigs, 1e-6)
            #print(eigs_new)
            for j in range(len(eigs_new)):
                data_arr.append([mu[t]/100, i, eigs_new[j].real, eigs_new[j].imag])
df = pd.DataFrame(np.array(data_arr), columns=['mu', 'conf_number', 're', 'im'])
print(df)
df.to_csv(path, index=False)

       mu  conf_number      re        im
0     0.0          1.0  0.0075  0.001230
1     0.0          1.0  0.0075  0.025057
2     0.0          1.0  0.0075  0.026069
3     0.0          1.0  0.0075  0.033522
4     0.0          1.0  0.0075  0.034583
...   ...          ...     ...       ...
9568  0.0        300.0  0.0075  0.078357
9569  0.0        300.0  0.0075  0.079360
9570  0.0        300.0  0.0075  0.080167
9571  0.0        300.0  0.0075  0.081715
9572  0.0        300.0  0.0075  0.083590

[9573 rows x 4 columns]


In [125]:
mu = [3, 5, 10]
file_number = [100, 100, 100, 100]
eig_number = [60, 60, 60, 60]

data_arr = []
#arr1 = np.empty(4)
smearing = 'smeared'
path = '/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/CSV/eigenvalues_mu.csv'

for t in range (len(mu)):
    w=mu[t]//10
    q=mu[t]-w*10
    for i in range (1, file_number[t]+1):
        a=i//1000
        b=(i-a*1000)//100
        c=(i-a*1000-b*100)//10
        d=(i-a*1000-b*100-c*10)
        if os.path.isfile('/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/time_32/mu0.%d%d/eigenvalues_eig=60_%d%d%d%d' % (w, q, a, b, c, d)):
            line = read_file('/home/ilya/lattice/eigenvalues/results/data/' + smearing + '/time_32/mu0.%d%d/eigenvalues_eig=60_%d%d%d%d' % (w, q, a, b, c, d))
            for k in range(1, 61):
                b=0
                if line[k][0] == '-':
                    b=1
                data_arr.append([mu[t]/100, i, float(line[k][0:22+b]), float(line[k][23+b:45+b])])
df = pd.DataFrame(np.array(data_arr), columns=['mu', 'conf_number', 're', 'im'])
print(df)
df.to_csv(path, index=False)

ValueError: Empty data passed with indices specified.